A notebook for making a 100-frame movie from 3D dust for Linnea as a test

Two environment variables need to be set in bash: DROPBOX_ROOT, and FFMPEG_ROOT. The DROPBOX_ROOT folder is the path to the top level directory of the dust-holgrams folder shared by Josh. E.g. in bash,

export DROPBOX_ROOT='/Users/catherinezucker/Dropbox/dust-holograms/'

You can also set the path to the FFMPEG executable:

export FFMPEG_PATH='/usr/local/bin'

Ideally, you would set these in your bash_profile profile  (now called zprofile in the latest OS operating systems). Alternatively, if you want to set one of these variables locally in this notebook, you can use:

```
import os
os.environ['DROPBOX_ROOT'] = '/Users/catherinezucker/Dropbox/dust-holograms'
os.environ['FFMPEG_ROOT'] = '/Usr/local/bin'

```

First thing we need to do is figure out how to write some json

In [17]:
import json
import numpy as np
import os
%matplotlib inline
#%matplotlib notebook
from matplotlib import pyplot as plt
from astropy import table

Josh is having some trouble with ```.zprofile``` so:

In [18]:
if (os.environ.keys() != 'DROPBOX_HOME'):
    os.environ['DROPBOX_ROOT'] = '/Users/jegpeek/Dropbox/dust-holograms'
    os.environ['FFMPEG_ROOT'] = '/Users/jegpeek/'

Using a variable called ```run_name``` to record everything we need about the run

In [19]:
def define_paths(run_name):
    if os.path.isdir("{}/{}/".format(os.environ['DROPBOX_ROOT'],run_name)) == False:
        os.mkdir("{}/{}/".format(os.environ['DROPBOX_ROOT'],run_name))
        os.mkdir("{}/{}/frames/".format(os.environ['DROPBOX_ROOT'],run_name))

This is the camera properties, which we will fix.

In [20]:
arc = False
if arc:
    ec = "szyz"
if not arc:
    ec = "rxyz"

t = True
cprops ={
    "projection": "stereographic",
    "step_size": 1,
    "max_dist": 500.0,
    "fov": 45.0,
    "x_pix": 60,
    "y_pix": 40,
    "vmax": "auto",
    "clip_mode": "tanh",
    "fuzzy": t,
    "randomize_angles": t,
    "euler_convention": ec}

In [21]:
print(cprops)

{'projection': 'stereographic', 'step_size': 1, 'max_dist': 500.0, 'fov': 45.0, 'x_pix': 60, 'y_pix': 40, 'vmax': 'auto', 'clip_mode': 'tanh', 'fuzzy': True, 'randomize_angles': True, 'euler_convention': 'rxyz'}


_In arc mode_ Linnea asked for 100 frames orbiting around a point with equal distance. We'll assume that the middle of these 100 frames is the Sun, and we'll set some sweep out angle in the Galactic Plane.

In [22]:
nframes = 19 # 100 frames
if arc:
    angle_sweep = 45 # half of this CCW and half CW from the sun

In [23]:
def sweep(xc, yc, angle_sweep, nframes):
    R = np.sqrt(xc**2+ yc**2)
    phi =np.arctan2(yc, xc)*180/np.pi
    dangs = np.linspace(0, angle_sweep, nframes)
    xs = xc - R*np.cos((phi-angle_sweep/2+dangs)*np.pi/180)
    ys = yc - R*np.sin((phi-angle_sweep/2+dangs)*np.pi/180)
    angs = (dangs-angle_sweep/2+phi)*np.pi/180 ## wait are these in RADIANs??
    return xs, ys, angs

In [24]:
def plot_ang(x0, y0, xs, ys, run_name):
    plt.figure(figsize=[5, 5])
    plt.scatter(x0, y0)
    plt.scatter(0, 0, c='red')
    plt.plot(xs, ys)
    plt.xlim([-500, 500])
    plt.ylim([-500, 500])
    plt.title(run_name)
    plt.savefig('{}/{}/arc_{}'.format(os.environ['DROPBOX_ROOT'],run_name,run_name))

In [25]:
def plot_track(xts, yts, xc, yc, run_name):
    plt.figure(figsize=[5, 5])
    plt.scatter(xts, yts)
    plt.scatter(0, 0, c='C1')
    plt.scatter(xc, yc, c='C2')
    plt.xlim([-500, 500])
    plt.ylim([-500, 500])
    plt.title(run_name)
    plt.savefig('{}/{}/arc_{}'.format(os.environ['DROPBOX_ROOT'],run_name,run_name))

In track mode we'll move from one point to another while pointing at the cloud and keeping to top of the camera perpindicular to the plane that contains the track and cloud

Given the current position of the viewer, the unit direction toward the start of the track, and the cloud position, compute the angles of rotation

In [26]:
def determine_angles(v_pos, c_pos, t_hat):
    x_hat = np.array([1, 0, 0])
    y_hat = np.array([0, 1, 0])
    z_hat = np.array([0, 0, 1])
    # the vector to the cloud
    c_vec = c_pos-v_pos
    # the unit vector toward the cloud
    c_hat = c_vec/np.sqrt(np.sum(c_vec**2))
    
    #FIRST
    # the unit vector perp to the plane containing the track and the cloud
    n_OCT = np.cross(c_hat, t_hat) #CHECK ORDER ####FLIPPED
    n_hat_OCT = n_OCT/np.sqrt(np.sum(n_OCT**2))
    #print('n_hat_OCT', n_hat_OCT)
    #print('c_hat', c_hat)
    #print('v_pos', v_pos)
    #print('c_vec', c_vec)
    #print('c_pos', c_pos)
    # find the new orientation of the side of the camera
    v_y_prime = np.cross(n_hat_OCT, x_hat) #CHECK ORDER (correct)
    v_hat_y_prime = v_y_prime/np.sqrt(np.sum(v_y_prime**2))
    # and find the angle to rotate first
    theta = np.arccos(np.dot(v_hat_y_prime,y_hat))
    #print('theta', theta)
    #print('v_hat_y_prime',v_hat_y_prime)
    
    #SECOND
    # the unit vector perp to the plane that contains the rotation 
    #n_phi = np.cross(v_hat_y_prime, x_hat) #CHECK ORDER ###FLIPPED
    #n_hat_phi = n_phi/np.sqrt(np.sum(n_phi**2))
    # the vector being rotated to
    #v_x_prime_prime = np.cross(n_hat_OCT,n_hat_phi) #CHECK ORDER (correct)
    #### OOOPS
    v_x_prime_prime = np.cross(v_hat_y_prime, n_hat_OCT)
    v_hat_x_prime_prime = v_x_prime_prime/np.sqrt(np.sum(v_x_prime_prime**2))
    # and the angle rotated
    phi = np.arccos(np.dot(v_hat_x_prime_prime, x_hat))
    #print('n_phi', n_phi)
    
    #FINALLY
    zeta = np.arccos(np.dot(v_hat_x_prime_prime, c_hat))
    print('c hat', c_hat)
    return theta, phi, zeta
    

given some inputs build a track

In [27]:
def track(xt1, yt1, zt1, xt2, yt2, zt2, nframes):
    xts = np.linspace(xt1, xt2, nframes)
    yts = np.linspace(yt1, yt2, nframes)
    zts = np.linspace(zt1, zt2, nframes)
    t_vec = np.array([xt2-xt1, yt2-yt1, zt2-zt1])
    t_hat = t_vec/np.sqrt(np.sum(t_vec**2))
    return xts, yts, zts, t_hat

In [28]:
def build_fprops(fprops, cprops, angs, xs, ys, zc):
    for i in range(nframes):
        fprops.append({
          "xyz": [xs[i], ys[i], zc],
          "angles": [angs[i], 0.0, 0.0]
        })
    allprops = {"camera_props": cprops,"frame_props":fprops }
    return allprops

In [29]:
def build_fprops_track(fprops, cprops, theatas, phis, zetas, xs, ys, zs):
    for i in range(nframes):
        fprops.append({
          "xyz": [xs[i], ys[i], zs[i]],
          "angles": [thetas[i], phis[i], zetas[i]]
        })
    allprops = {"camera_props": cprops,"frame_props":fprops }
    return allprops

Let's read in a list of molecular clouds and make movies for each one:

In [30]:
clouds = table.Table.read('{}/Holo_Cloud_Targets.csv'.format(os.environ['DROPBOX_ROOT']))
#clouds = table.Table.read('{}/Holo_TEST_Targets.csv'.format(os.environ['DROPBOX_ROOT']))

In [ ]:
makeimages = True
makemovies = True
oldzeta = 0
for c in clouds:
    print(c['cloud'])
    run_name = c['cloud'] + '_test_track_2'
    define_paths(run_name)
    if arc:
        xs, ys, angs = sweep(c['x'], c['y'], angle_sweep, nframes)
        print(xs, ys, angs)
        plot_ang(c['x'], c['y'], xs, ys, run_name)
        # we make an empty list of frames to which we can append frames
        fprops = []
        allprops = build_fprops(fprops, cprops, angs, xs, ys, np.float(c['z']))
    if not arc:
        #hardcoding a track
        #xts, yts, zts, t_hat = track(c['xt1'], c['yt1'], c['zt1'], c['xt2'], c['yt2'], c['zt2'], nframes)
        xts, yts, zts, t_hat = track(c['y']*0.5, c['x']*(-0.5), 0, c['y']*(-0.5), c['x']*(0.5), 0, nframes)
        plot_track(xts, yts, c['x'], c['y'], run_name)
        print(t_hat)
        fprops = []
        thetas = np.zeros(nframes)
        phis = np.zeros(nframes)
        zetas = np.zeros(nframes)
        for j in np.arange(nframes):
            v_pos = np.array([xts[j], yts[j], zts[j]])
            c_pos = np.array([c['x'], c['y'], c['z']])
            thetas[j], phis[j], zetas[j] = determine_angles(v_pos, c_pos, t_hat)
            print(thetas[j]*180/np.pi, phis[j]*180/np.pi, zetas[j]*180/np.pi,(zetas[j]-oldzeta)*180/np.pi )
            oldzeta = zetas[j]
        allprops = build_fprops_track(fprops, cprops, thetas, phis, zetas, xts, yts, zts)

    with open('{}/{}/{}.json'.format(os.environ['DROPBOX_ROOT'],run_name,run_name), 'w') as outfile:
        json.dump(allprops, outfile,indent=2)
    if makeimages:
        os.system("python3 project_frames.py {}/{}/{}.json {}/leike2020_bayestar19_splice.npy {}/{}/frames/{}_{{:05d}}.png"
          .format(os.environ['DROPBOX_ROOT'],run_name,run_name,os.environ['DROPBOX_ROOT'],os.environ['DROPBOX_ROOT'],run_name, run_name))
    if makemovies:
        os.system("{}/ffmpeg -y -r 30 -start_number 0 -i {}/{}/frames/{}_%05d.png -c:v libx264 -s 600x400 -r 30 -pix_fmt yuv420p {}/{}/{}.mp4"
          .format(os.environ['FFMPEG_ROOT'],os.environ['DROPBOX_ROOT'],run_name,run_name,os.environ['DROPBOX_ROOT'],run_name, run_name))


Chamaeleon
[0.87415728 0.48564293 0.        ]
c hat [ 0.79961764 -0.54710681 -0.24755964]
165.9787201722434 7.666008811742404 36.21277000968203 36.21277000968203
c hat [ 0.77291301 -0.58207029 -0.25258595]
165.9787201722434 7.666008811742404 38.75011723401205 2.537347224330018
c hat [ 0.74355323 -0.61719778 -0.25728485]
165.9787201722434 7.666008811742404 41.38721154991796 2.6370943159059017
c hat [ 0.71148843 -0.65219773 -0.26157663]
165.9787201722434 7.666008811742355 44.118241938244594 2.7310303883266402
c hat [ 0.67672133 -0.68674683 -0.26538091]
165.9787201722434 7.666008811742404 46.93527873071644 2.817036792471846
c hat [ 0.63931583 -0.72049879 -0.2686201 ]
165.9787201722434 7.666008811742404 49.82823348409732 2.892954753380875
c hat [ 0.59940312 -0.75309622 -0.27122313]
165.9787201722434 7.666008811742404 52.7849310296664 2.9566975455690936
c hat [ 0.5571841  -0.78418504 -0.27312946]
165.9787201722434 7.666008811742355 55.79130832473395 3.006377295067542
c hat [ 0.51292742 -0.8

100% (19 of 19) |########################| Elapsed Time: 0:00:04 ETA:  00:00:00

All workers done.
0.397824734759316
Loading map ...
Ray-casting frames ...
frame 0: vmax = 0.005471662555777584
frame 1: vmax = 0.005841045070243126
frame 2: vmax = 0.005940742924656661
frame 3: vmax = 0.004845269660880149
frame 4: vmax = 0.0046153210579832375
frame 5: vmax = 0.004162846945910133
frame 6: vmax = 0.004352928346506814
frame 7: vmax = 0.004084615535386547
frame 8: vmax = 0.004229031643964845
frame 9: vmax = 0.0045620662122673825
frame 10: vmax = 0.004615446880634409
frame 11: vmax = 0.003986366011784412
frame 12: vmax = 0.0038051796982472297
frame 13: vmax = 0.004631778863418731
frame 14: vmax = 0.004224798836025002
frame 15: vmax = 0.0042838267816187
frame 16: vmax = 0.005957081943997764
frame 17: vmax = 0.006169471150453319
frame 18: vmax = 0.005513385313402978
Worker finished.


ffmpeg version 5.0-tessus  https://evermeet.cx/ffmpeg/  Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.17)
  configuration: --cc=/usr/bin/clang --prefix=/opt/ffmpeg --extra-version=tessus --enable-avisynth --enable-fontconfig --enable-gpl --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libfreetype --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libmysofa --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvmaf --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs --enable-libxvid --enable-libzimg --enable-libzmq --enable-libzvbi --enable-version3 --pkg-config-flags=--static --disable-ffplay

Ophiuchus
[0.07728751 0.99700885 0.        ]
c hat [0.89438236 0.36375999 0.26030531]
178.66547321008858 16.722273735393927 20.952807428524604 -66.61341459226995
c hat [0.90862956 0.32229491 0.26555285]
178.66547321008858 16.722273735393927 18.422571877184208 -2.530235551340394
c hat [0.92155562 0.27854712 0.27045654]
178.66547321008858 16.722273735393927 15.793595759785687 -2.628976117398521
c hat [0.93289439 0.23263613 0.27493361]
178.66547321008858 16.722273735393927 13.071681291663333 -2.721914468122353
c hat [0.94238251 0.18474193 0.27890073]
178.66547321008858 16.722273735393905 10.264716266008953 -2.806965025654379
c hat [0.94977127 0.13510718 0.28227749]
178.66547321008858 16.722273735393927 7.382710613442561 -2.8820056525663937
c hat [0.95483952 0.08403562 0.28499037]
178.66547321008858 16.722273735393927 4.437723120093873 -2.944987493348688
c hat [0.95740671 0.0318863  0.28697674]
178.66547321008858 16.722273735393927 1.4436643277933123 -2.9940587923005606
c hat [ 0.95734473 

...

Musca
[0.85749293 0.51449576 0.        ]
c hat [ 0.83455826 -0.53108253 -0.14650552]
171.91871539561538 4.821298199070023 33.12066790338364 -99.50147870145769
c hat [ 0.80920182 -0.56816298 -0.14961031]
171.91871539561538 4.821298199070023 35.700817959839924 2.5801500564562834
c hat [ 0.78106122 -0.60554185 -0.15252029]
171.91871539561538 4.821298199070023 38.386971804392374 2.6861538445524573
c hat [ 0.75005859 -0.64290736 -0.15518454]
171.91871539561538 4.821298199070023 41.1732846361977 2.7863128318053216
c hat [ 0.71617106 -0.67990924 -0.15755138]
171.91871539561546 4.821298199070023 44.0515840778228 2.878299441625097
c hat [ 0.67944142 -0.71616799 -0.15957059]
171.91871539561538 4.821298199070023 47.0113056256742 2.9597215478514003
c hat [ 0.63998609 -0.75128802 -0.1611959 ]
171.91871539561538 4.821298199070023 50.03955523414622 3.028249608472024
c hat [ 0.59799922 -0.78487398 -0.16238772]
171.91871539561538 4.821298199070023 53.1213181476744 3.0817629135281783
c hat [ 0.55375167 

100% (19 of 19) |########################| Elapsed Time: 0:00:04 ETA:  00:00:00

All workers done.
0.397824734759316
Loading map ...
Ray-casting frames ...
frame 0: vmax = 0.006188358129355038
frame 1: vmax = 0.006840469747094176
frame 2: vmax = 0.005096695690070192
frame 3: vmax = 0.004659132819964725
frame 4: vmax = 0.0058390891110902884
frame 5: vmax = 0.004326851787758642
frame 6: vmax = 0.00421684138225828
frame 7: vmax = 0.004790566987776401
frame 8: vmax = 0.005031823955279833
frame 9: vmax = 0.005040003767589951
frame 10: vmax = 0.0073867325289393194
frame 11: vmax = 0.005298755483578134
frame 12: vmax = 0.005414246217434993
frame 13: vmax = 0.005686232556341565
frame 14: vmax = 0.007880499556289578
frame 15: vmax = 0.0049518009346065815
frame 16: vmax = 0.005583321185637033
frame 17: vmax = 0.005624149695911911
frame 18: vmax = 0.006572558078361908
Worker finished.


ffmpeg version 5.0-tessus  https://evermeet.cx/ffmpeg/  Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.17)
  configuration: --cc=/usr/bin/clang --prefix=/opt/ffmpeg --extra-version=tessus --enable-avisynth --enable-fontconfig --enable-gpl --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libfreetype --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libmysofa --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvmaf --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs --enable-libxvid --enable-libzimg --enable-libzmq --enable-libzvbi --enable-version3 --pkg-config-flags=--static --disable-ffplay

Pipe
[-0.03309444  0.99945223  0.        ]
c hat [0.87760041 0.47573625 0.05909767]
179.87449468177837 3.7847431394564848 28.415656490685624 -57.217712452600864
c hat [0.89822855 0.43536246 0.06037384]
179.87449468177837 3.7847431394564848 25.816710398254067 -2.5989460924315586
c hat [0.91775461 0.39234608 0.06157131]
179.87449468177837 3.7847431394564848 23.108901236871805 -2.707809161382264
c hat [0.93585509 0.34676773 0.06266887]
179.87449468177837 3.7847431394564848 20.298077257144744 -2.810823979727057
c hat [0.9521982  0.29877738 0.06364489]
179.87449468177837 3.7847431394564848 17.39251160767566 -2.9055656494690836
c hat [0.96645785 0.24859961 0.06447829]
179.87449468177837 3.7847431394564848 14.402979375786105 -2.989532231889559
c hat [0.97833003 0.19653467 0.06514961]
179.87449468177837 3.7847431394564848 11.342700411005392 -3.060278964780711
c hat [0.98755012 0.14295408 0.06564218]
179.87449468177837 3.7847431394564848 8.227126497702693 -3.1155739133026996
c hat [0.9939096  0

In [ ]:
plt.scatter(xts, yts)
plt.scatter()

In [ ]:
print(nframes)